In [1]:
import warnings
import logging
warnings.filterwarnings("ignore")

In [2]:
import copy
import datetime
import glob
import warnings
from functools import lru_cache, partial
from pprint import pprint

import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ray
import torch
import os

from gym_trading_env.renderer import Renderer
from IPython.display import display
from environments.environments import NeuralForecastingTradingEnv
# from ray import train, tune
from tqdm.autonotebook import tqdm
from utils.utils import build_market_image

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from typing import Dict, Tuple
from pprint import pprint

from neuralforecast.core import NeuralForecast


from ray.rllib.algorithms.callbacks import DefaultCallbacks

from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.env import BaseEnv
from ray.rllib.evaluation import Episode, RolloutWorker

from ray.rllib.policy import Policy
from ray.tune import JupyterNotebookReporter

from ray.tune.registry import register_env
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search.optuna import OptunaSearch

In [3]:
%%writefile custom.css
/* custom.css */

/* Set background color for all widgets */
.tuneStatus {
    /* white background */
    background-color: #ffffff;
    
    width: 100%;
    height: 100%;
    overflow:'auto',
    transition:'all'
}

Overwriting custom.css


In [4]:
from IPython.display import display, HTML
import tensorboard

# Load custom CSS file
css = HTML('<link rel="stylesheet" type="text/css" href="./custom.css">') # make sure the path is correct, it fails silently otherwise
display(css)

In [5]:
COIN_PAIRS=['BTC/USDT','ETH/USDT','SOL/USDT','BNB/USDT','XRP/USDT','ADA/USDT',
            'ETH/BTC','SOL/ETH','BNB/ETH','XRP/ETH',"ADA/ETH",
            'SOL/BTC','SOL/BNB',
            'XRP/BTC','XRP/BNB',
            'ADA/BTC','ADA/BNB',
            ]
target_pair='ETHUSDT'
time_frame="1h"

In [6]:
model=NeuralForecast.load("forecasting_model/")

Seed set to 5
Seed set to 11
Seed set to 17
Seed set to 6


In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:

symbol='ETH/USDT'
data=build_market_image(target_pair=symbol,time_frame='1h',axis=0,verbose=1,only_target=True)
data=data[data['symbol']==target_pair.replace('/','')].copy()
data['ds']=data.index



['data/binanceus-ETHUSDT-1h.pkl']


0it [00:00, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

In [9]:
split_date=datetime.datetime(year= 2024, month= 3, day=1)
end_date=split_date+datetime.timedelta(days=14)

symbol='ETHUSDT'
hf_data=data.copy()

hf_train_data=hf_data.groupby('symbol').apply(lambda x: x[:split_date])
hf_test_data=hf_data.groupby('symbol').apply(lambda x: x[split_date:end_date])

hf_train_data=hf_train_data.reset_index(level=0,drop=True).reset_index()
hf_test_data=hf_test_data.reset_index(level=0,drop=True).reset_index()


trade_data=data[data['symbol']==symbol].copy()
train_data=trade_data[:split_date]
test_data=trade_data[split_date:end_date]
# hf_test_data

In [10]:
train_data.head(5)

,open,high,low,close,volume,y,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,...,feature_WILLIAMS_FRACTAL_BearishFractal,feature_WILLIAMS_FRACTAL_BullishFractal,feature_9_period_WMA,feature_WOBV,feature_WTO_WT1,feature_WTO_WT2,feature_26_period_ZLEMA,ds,unique_id,symbol
date_close,,,,,,,,,,,,,,,,,,,,,
2022-01-01 10:00:00,3724.17,3729.49,3710.60,3716.00,19.49929,3716.00,-0.065217,0.333333,-0.5,-0.5,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,2022-01-01 10:00:00,8,ETHUSDT
2022-01-01 11:00:00,3717.23,3730.92,3675.05,3698.68,62.84954,3698.68,-0.021739,0.333333,-0.5,-0.5,...,0.0,0.0,0.0,-1088.554033,-121.212121,0.000000,0.0,2022-01-01 11:00:00,8,ETHUSDT
2022-01-01 12:00:00,3691.28,3711.32,3691.28,3692.08,24.00284,3692.08,0.021739,0.333333,-0.5,-0.5,...,0.0,0.0,0.0,-1246.972777,-102.124690,0.000000,0.0,2022-01-01 12:00:00,8,ETHUSDT
2022-01-01 13:00:00,3696.63,3717.22,3695.50,3717.22,15.28388,3717.22,0.065217,0.333333,-0.5,-0.5,...,0.0,0.0,0.0,-862.736034,-46.566957,0.000000,0.0,2022-01-01 13:00:00,8,ETHUSDT
2022-01-01 14:00:00,3716.53,3724.24,3708.17,3710.01,27.37488,3710.01,0.108696,0.333333,-0.5,-0.5,...,0.0,0.0,0.0,-1060.108918,-11.818867,-70.430659,0.0,2022-01-01 14:00:00,8,ETHUSDT


In [11]:
def prep_forecasts(df):
    forecast_array=[]
    # print(self.df.columns)

    model.dataset, model.uids, model.last_dates, model.ds=model._prepare_fit(df[['ds','unique_id','y']],
                                                                                        static_df=None, 
                                                                                        sort_df=None,
                                                                                        predict_only=False,
                                                                                        id_col='unique_id', 
                                                                                        time_col='ds', 
                                                                                        target_col='y')
    forecasts=model.predict_insample()
    forecasts_series=forecasts.groupby('cutoff').apply(lambda x: x.select_dtypes(np.number).values.flatten())
    new_df=df[df['ds'].isin([c for c in forecasts_series.index])]
    forecasts_series=forecasts_series[new_df.index]
    forecast_array=[c for c in forecasts_series]
    return forecast_array,new_df

In [12]:
os.environ['PYTORCH_ENABLE_MPS_FALLBACK']="1"
train_forecast_array,train_df=prep_forecasts(train_data)
test_forecast_array,test_df=prep_forecasts(test_data)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
def rew_func(history):
    portfolio_initial_value= history["portfolio_valuation",0]
    current_position=history['position',-1]
    portfolio_return=history["portfolio_valuation"]
    portfolio_pct_change=pd.Series(np.array(portfolio_return)).pct_change().fillna(0)
    # print(portfolio_pct_change)
    reward = (portfolio_pct_change.mean() / portfolio_pct_change.std())
    
    # print(np.isnan(reward),reward)
    reward = 0 if np.isnan(reward) else reward
    # print(np.isnan(reward),reward)
    return float(reward)


In [14]:
train_env_config=dict(
                name='ETHUSDT_train',
                forecast_horizon=7,
                context_length=35,
                reward_function=rew_func,
                forecasts=train_forecast_array,
                df = train_df, # Your dataset with your custom features
                positions = [ -1, -.5, 0, .5 , 1], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
                trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
                borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
                max_episode_duration=168,
                verbose=0
                )

test_env_config=copy.deepcopy(train_env_config)
test_env_config['name']='ETHUSDT_test'
test_env_config['df'] = test_df
test_env_config['forecasts']=test_forecast_array



In [15]:
train_env=NeuralForecastingTradingEnv(**train_env_config)

In [16]:
obs,info=train_env.reset()
obs.shape

(155,)

In [17]:
train_env.df.tail(5)

,open,high,low,close,volume,y,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,...,feature_9_period_WMA,feature_WOBV,feature_WTO_WT1,feature_WTO_WT2,feature_26_period_ZLEMA,ds,unique_id,symbol,dynamic_feature__0,dynamic_feature__1
date_close,,,,,,,,,,,,,,,,,,,,,
2024-02-29 14:00:00,3463.10,3488.78,3449.92,3481.07,34.3386,3481.07,0.108696,0.0,0.433333,-0.338356,...,3478.406000,-1.616240e+06,55.314524,58.078706,3495.087080,2024-02-29 14:00:00,8,ETHUSDT,0,0
2024-02-29 15:00:00,3480.20,3495.67,3451.94,3466.28,25.1406,3466.28,0.152174,0.0,0.433333,-0.338356,...,3476.492889,-1.616612e+06,52.310426,56.521387,3497.598408,2024-02-29 15:00:00,8,ETHUSDT,0,0
2024-02-29 16:00:00,3469.53,3486.26,3458.12,3467.35,46.0573,3467.35,0.195652,0.0,0.433333,-0.338356,...,3474.723111,-1.616563e+06,49.325539,53.755590,3500.128896,2024-02-29 16:00:00,8,ETHUSDT,0,0
2024-02-29 17:00:00,3467.35,3469.88,3397.82,3441.86,110.3305,3441.86,0.239130,0.0,0.433333,-0.338356,...,3468.167556,-1.619375e+06,39.269433,49.054980,3496.173422,2024-02-29 17:00:00,8,ETHUSDT,0,0
2024-02-29 18:00:00,3441.94,3452.31,3355.12,3386.75,106.6427,3386.75,0.282609,0.0,0.433333,-0.338356,...,3451.106889,-1.625252e+06,25.213777,41.529794,3482.412428,2024-02-29 18:00:00,8,ETHUSDT,0,0


In [18]:
obs=train_env.reset()
terminated = truncated = False
# env.action_space.sa
while not terminated and not truncated:
    action = train_env.action_space.sample()
    obs, reward, terminated, truncated, info = train_env.step(action)
info['reward']
train_env.results_metrics
info

{'idx': 4881,
 'step': 167,
 'date': numpy.datetime64('2022-07-23T19:00:00.000000000'),
 'position_index': 2,
 'position': 0,
 'real_position': 0.0,
 'data_symbol': 'ETHUSDT',
 'data_close': 1498.78,
 'data_volume': 1143.63129,
 'data_ds': Timestamp('2022-07-23 19:00:00'),
 'data_y': 1498.78,
 'data_unique_id': 8,
 'data_low': 1488.5,
 'data_open': 1501.19,
 'data_high': 1514.68,
 'portfolio_valuation': 982.3861843359693,
 'portfolio_distribution_asset': 0,
 'portfolio_distribution_fiat': 982.3861843359693,
 'portfolio_distribution_borrowed_asset': 0,
 'portfolio_distribution_borrowed_fiat': 0,
 'portfolio_distribution_interest_asset': 0.0,
 'portfolio_distribution_interest_fiat': 0.0,
 'reward': -0.00853396419862617}

In [19]:


register_env("train_env", lambda config: NeuralForecastingTradingEnv(**train_env_config))
register_env("test_env", lambda config: NeuralForecastingTradingEnv(**test_env_config))
register_env("test_env", lambda config: gym.make('TradingEnv'))


In [20]:

def trial_str_creator(trial):
    return "{}_{}_trading_agent".format(trial.trainable_name, trial.trial_id)



In [21]:
# type(float(np.float64(1660)))

In [22]:
class TraderCallbacks(DefaultCallbacks):
    
    def on_episode_start(
            self,
            *,
            worker: RolloutWorker,
            base_env: BaseEnv,
            policies: Dict[str, Policy],
            episode: Episode,
            env_index: int,
            **kwargs,
        ):
        # Make sure this episode has just been started (only initial obs
        # logged so far).


        # Create lists to store angles in
        episode.hist_data["portfolio_values"] = []
        episode.hist_data["positions"] = []

    def on_episode_step(
        self,
        *,
        worker: RolloutWorker,
        base_env: BaseEnv,
        policies: Dict[str, Policy],
        episode: Episode,
        env_index: int,
        **kwargs,
    ):
        # Make sure this episode is ongoing.

        portfolio_valuation = episode.last_info_for()['portfolio_valuation']
        position=episode.last_info_for()['position']
        episode.hist_data["portfolio_values"].append(portfolio_valuation)
        episode.hist_data["positions"].append(position)

        
    def on_episode_end(
        self,
        *,
        worker: RolloutWorker,
        
        base_env: BaseEnv,
        policies: Dict[str, Policy],
        episode: Episode,
        env_index: int,
        **kwargs,
    ):


        portfolio_valuation = episode.hist_data["portfolio_values"][-1]
        position_changes = np.sum(np.diff(episode.hist_data["positions"]) != 0)
        # m_port=np.mean(position_changes)
        
        episode.custom_metrics["portfolio_value"] = float(portfolio_valuation)
        episode.custom_metrics["position_changes"] = float(position_changes)
        # train.report(episode.custom_metrics)
        


In [23]:

ppo_algo = (PPOConfig().training(lr=ray.tune.loguniform(1e-8, 1e-5),
                         gamma=ray.tune.loguniform(.8, .999),      
                         lambda_=ray.tune.loguniform(.9, 1),
                         vf_loss_coeff=ray.tune.loguniform(.5, 1),     
                         kl_coeff=ray.tune.loguniform(.003,.3),
                         num_sgd_iter=ray.tune.randint(3,30),
                         clip_param=ray.tune.choice([.1,.2,.3]),
                         model={
                             'fcnet_hiddens': ray.tune.choice([[128,128],[256,256],[512,512],[1024,1024]]),
                            #  'use_attention':True,#tune.choice([True,False]),

                                },

                         )

                        .resources(
                                    num_gpus=0,
                                    
                                    )
                        .callbacks(TraderCallbacks)
                        .reporting(keep_per_episode_custom_metrics=False)
                        .environment(env="train_env")
                        .debugging(log_level='ERROR')
                            
                        .evaluation(
                                    evaluation_config={

                                                        "env": 'test_env',  # The dictionary we built before (only the overriding keys to use in evaluation)
                                                        "explore": False,
                                                        }
                                    )
                        
            
            )

ppo_algo_config=ppo_algo.to_dict()
ppo_algo_config.pop('num_gpus_per_worker')
ppo_algo_config.pop('num_cpus_per_worker')
ppo_algo_config.pop('num_cpus_per_learner_worker')
ppo_algo_config.pop('num_gpus_per_learner_worker')
ppo_algo_config.pop('num_learner_workers')
ppo_algo_config['num_cpus_per_env_runner']=2
ppo_algo_config['num_gpus_per_env_runner']=0

ppo_algo_config['num_cpus_per_learner']=1
ppo_algo_config['num_gpus_per_learner']=1
pprint(ppo_algo_config)

{'_AlgorithmConfig__prior_exploration_config': None,
 '_disable_action_flattening': False,
 '_disable_execution_plan_api': -1,
 '_disable_initialize_loss_from_dummy_batch': False,
 '_disable_preprocessor_api': False,
 '_enable_rl_module_api': -1,
 '_env_to_module_connector': None,
 '_evaluation_parallel_to_training_wo_thread': False,
 '_fake_gpus': False,
 '_is_atari': None,
 '_learner_class': None,
 '_learner_connector': None,
 '_model_config_dict': {},
 '_module_to_env_connector': None,
 '_per_module_overrides': {},
 '_rl_module_spec': None,
 '_run_training_always_in_thread': False,
 '_tf_policy_handles_more_than_one_loss': False,
 'action_mask_key': 'action_mask',
 'action_space': None,
 'actions_in_input_normalized': False,
 'add_default_connectors_to_env_to_module_pipeline': True,
 'add_default_connectors_to_learner_pipeline': True,
 'add_default_connectors_to_module_to_env_pipeline': True,
 'algorithm_config_overrides_per_module': {},
 'always_attach_evaluation_results': True,
 '

In [24]:
# ppo_algo.build()

In [25]:
# First, get the handle for the logger you want to modify
loggers=["ray.data","ray.tune","ray.rllib","ray.train","ray.serve","ray"]
for logger in loggers:
    ray_logger=logging.getLogger(logger)
    ray_logger .setLevel(logging.ERROR)

In [26]:
logger = logging.getLogger("ray")
logger.setLevel(logging.ERROR)

In [27]:
ray.init(include_dashboard=False,
        #  _temp_dir='~/Users/sebastiancoombs/Documents/Git/MultiTrader/Rayrl_tune/tmp',
        #  dashboard_host="0.0.0.0"
         )

2024-10-02 10:36:09,494	INFO worker.py:1749 -- Started a local Ray instance.


Python version:,3.10.13
Ray version:,2.23.0


In [28]:
checkpoint_path='/Users/sebastiancoombs/Documents/Git/MultiTrader/Agent/checkpoints'

checkpoint_config=ray.train.CheckpointConfig(checkpoint_at_end=True,
                                         checkpoint_score_attribute='env_runners/custom_metrics/portfolio_value_mean',
                                        #  checkpoint_score_attribute='env_runners/episode_reward_mean',
                                        #  num_to_keep=1,
                                        #  checkpoint_frequency=10,
                                         )
reporter= JupyterNotebookReporter(
    sort_by_metric=True,
    metric='env_runners/custom_metrics/portfolio_value_max',
    mode='max',


    metric_columns={
                    'training_iteration':'iter',
                    'env_runners/reward':'reward',
                    'env_runners/custom_metrics/portfolio_value_mean': 'account_value_mean',
                    'env_runners/custom_metrics/position_changes_mean': 'n_trades_mean',
                    'env_runners/episode_reward_mean':'sharpe_ratio_mean',

                    'env_runners/custom_metrics/portfolio_value_max':'account_value_max',
                    'env_runners/custom_metrics/portfolio_value_min': 'account_value_min',

                    'env_runners/custom_metrics/position_changes_max': 'n_trades_max',
                    
                    'env_runners/episode_reward_max': 'sharpe_ratio_max',
                    'env_runners/episode_reward_min':'sharpe_ratio_min',
                    'env_runners/custom_metrics/portfolio_value_min': 'account_value_min',
                    

    },
    parameter_columns={

                        'lr':'lr',
                        'gamma':'gamma',
                        'lambda_':'lambda',
                        'kl_coeff':'kl_coeff',
                        'vf_loss_coeff':'vf_loss_coeff',
                        'num_sgd_iter':'num_sgd_iter',
                        'model/fcnet_hiddens':'fcnet_hiddens',
                        # 'model/use_attention':'use_attention'
                        }
    )

run_config=ray.train.RunConfig(

                            stop={
                                "training_iteration": 500,
                                #   "evaluation/episode_reward_mean": 1.5,
                                #   "timesteps_total":5000,
                                },
                                
                            
                            storage_path=checkpoint_path,
                            checkpoint_config=checkpoint_config,
                            progress_reporter=reporter

                        )

# search_algo = OptunaSearch(
#                             mode="max",                            
#                             metric='env_runners/custom_metrics/portfolio_value_mean',
# )



search_algo = HyperOptSearch(
    metric='env_runners/custom_metrics/portfolio_value_mean',
                            #  metric='episode_reward_mean',
                            
                             mode='max',
                             )

search_algo= ConcurrencyLimiter(
                                search_algo, 
                                max_concurrent=2
                                )

tune_config=ray.tune.TuneConfig(num_samples=20,
                            reuse_actors=False,
                            search_alg=search_algo,
                            trial_name_creator=trial_str_creator,
                            trial_dirname_creator=trial_str_creator,

                            )
css = HTML('<link rel="stylesheet" type="text/css" href="./custom.css">') # make sure the path is correct, it fails silently otherwise
display(css)
tuner = ray.tune.Tuner(
    "PPO",
    param_space=ppo_algo_config,
    tune_config=tune_config,
    run_config=run_config,
    )
appo_results=tuner.fit()

(PPO pid=46372) Install gputil for GPU system monitoring.
(PPO pid=46491) Install gputil for GPU system monitoring.
(PPO pid=46491) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/sebastiancoombs/Documents/Git/MultiTrader/Agent/checkpoints/PPO_2024-10-02_10-36-10/PPO_9494c3f0_trading_agent/checkpoint_000000)
(PPO pid=75842) Install gputil for GPU system monitoring.
(PPO pid=46372) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/sebastiancoombs/Documents/Git/MultiTrader/Agent/checkpoints/PPO_2024-10-02_10-36-10/PPO_a249e287_trading_agent/checkpoint_000000)
(PPO pid=78298) Install gputil for GPU system monitoring.
(PPO pid=78298) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/sebastiancoombs/Documents/Git/MultiTrader/Agent/checkpoints/PPO_2024-10-02_10-36-10/PPO_d48a14c0_trading_agent/checkpoint_000000)
(PPO pid=94342) Install gputil for GPU system monitoring.
(PPO pid=75842) Checkpoint successfully c

In [29]:


# appo_results=tuner.fit()

In [30]:
df_results = appo_results.get_dataframe()
df_results.to_csv('Rayrl_tune/results/appo_tune.csv')
# df_results

In [31]:

custom_cols=[
                'config/lr',
                'config/gamma',
                'config/lambda',
                'config/kl_coeff',
                'config/model/fcnet_hiddens',

                ]
custom_cols+=[c for c in df_results.filter(like='episode_reward').columns]
custom_cols+=[c for c in df_results.filter(like='custom_metrics').columns]
custom_cols=[c for c in custom_cols if 'sampler_results' not in c ]
custom_cols=[c for c in custom_cols if 'hist_stats' not in c ]
custom_cols=list(set(custom_cols))

In [32]:

df_results[custom_cols].sort_values('env_runners/custom_metrics/portfolio_value_mean',ascending=False)


,env_runners/episode_reward_min,env_runners/custom_metrics/position_changes_min,env_runners/custom_metrics/portfolio_value_min,config/gamma,config/lambda,env_runners/custom_metrics/position_changes_max,config/keep_per_episode_custom_metrics,env_runners/custom_metrics/portfolio_value_max,config/lr,config/kl_coeff,env_runners/custom_metrics/portfolio_value_mean,env_runners/episode_reward_max,config/model/fcnet_hiddens,env_runners/episode_reward_mean,env_runners/custom_metrics/position_changes_mean
7,-32.762317,0.0,816.446893,0.907288,0.931735,100.0,False,1235.359794,3.968898e-06,0.045623,1012.330840,36.710575,"(256, 256)",3.672888,29.22
19,-28.248196,61.0,836.779949,0.991647,0.938133,106.0,False,1420.565318,9.042351e-07,0.014064,1010.809756,48.462795,"(1024, 1024)",1.651702,83.24
1,-26.383891,0.0,830.606432,0.801419,0.905119,51.0,False,1271.652529,8.678756e-06,0.007992,1007.819985,37.862446,"(512, 512)",2.060532,10.65
9,-42.732446,0.0,840.419069,0.886691,0.936111,65.0,False,1304.073041,4.252308e-06,0.012700,1006.729525,56.962839,"(1024, 1024)",2.007474,22.26
6,-30.958084,0.0,700.870566,0.962928,0.921318,97.0,False,1394.166110,6.553096e-06,0.066770,1005.106935,38.170927,"(512, 512)",1.522289,23.24
13,-39.413673,58.0,694.072973,0.812705,0.978985,120.0,False,1437.518616,1.805395e-06,0.151590,1002.217227,67.437822,"(256, 256)",2.101667,86.68
3,-52.027254,116.0,777.996390,0.980304,0.917802,142.0,False,1434.513210,3.473209e-06,0.007207,1001.016048,43.795167,"(512, 512)",-3.743820,128.81
17,-56.433539,117.0,795.565719,0.899074,0.918939,144.0,False,1173.793730,2.277465e-08,0.021118,990.490028,32.772512,"(1024, 1024)",-3.226908,132.16
4,-31.715616,16.0,787.483152,0.902111,0.987774,115.0,False,1247.827358,3.993720e-06,0.033347,990.050779,39.066642,"(512, 512)",-0.662061,45.39
8,-34.413027,121.0,834.893841,0.974853,0.977220,143.0,False,1122.686543,6.721118e-08,0.006032,987.881310,37.324448,"(1024, 1024)",-1.265881,133.24


In [33]:
# [c for c in df_results.columns]

In [34]:
best=appo_results.get_best_result(metric='env_runners/custom_metrics/portfolio_value_mean', mode='max')
best.checkpoint

Checkpoint(filesystem=local, path=/Users/sebastiancoombs/Documents/Git/MultiTrader/Agent/checkpoints/PPO_2024-10-02_10-36-10/PPO_b39561b5_trading_agent/checkpoint_000000)

In [35]:
from ray.rllib.algorithms.algorithm import Algorithm

In [36]:
agent= Algorithm.from_checkpoint(best.checkpoint.path)

In [37]:
num_epochs=1000
for i in tqdm(range(num_epochs)):
    agent.train()
    if i % 100 == 0:
        checkpoint_dir = agent.save('Agent/final_checkpoints').checkpoint.path
        print(f"Checkpoint saved in directory {checkpoint_dir}")

  0%|          | 0/1000 [00:00<?, ?it/s]

Checkpoint saved in directory Agent/final_checkpoints
Checkpoint saved in directory Agent/final_checkpoints
Checkpoint saved in directory Agent/final_checkpoints
Checkpoint saved in directory Agent/final_checkpoints
Checkpoint saved in directory Agent/final_checkpoints
Checkpoint saved in directory Agent/final_checkpoints
Checkpoint saved in directory Agent/final_checkpoints


In [ ]:
test_env=NeuralForecastingTradingEnv(**test_env_config)

In [ ]:
obs,info=train_env.reset()
action=None
reward=0
terminated = truncated = False

while not terminated and not truncated:

    action = agent.compute_single_action(obs,explore=False)
    action = train_env.action_space.sample()
    # print(action)
    next_obs, reward, terminated, truncated, info = train_env.step(action)

    obs=next_obs
info
train_env.get_metrics(),info['portfolio_valuation']

In [ ]:
eval_profit=[]
for i in tqdm( range(100)):
    obs,info=train_env.reset()
    action=None
    reward=0
    terminated = truncated = False

    while not terminated and not truncated:

        action = agent.compute_single_action(obs,explore=False)
        # action = train_env.action_space.sample()
        # print(action)
        next_obs, reward, terminated, truncated, info = train_env.step(action)

        obs=next_obs

    eval_profit.append(info['portfolio_valuation'])
    info['portfolio_valuation']
profits=[e for e in eval_profit]
plt.plot(profits)
np.mean(profits)

In [ ]:
eval_profit=[]
for i in tqdm( range(100)):
    obs,info=test_env.reset()
    action=None
    reward=0
    terminated = truncated = False

    while not terminated and not truncated:

        action = agent.compute_single_action(obs,explore=False)
        # action = train_env.action_space.sample()
        # print(action)
        next_obs, reward, terminated, truncated, info = test_env.step(action)

        obs=next_obs

    eval_profit.append(info['portfolio_valuation'])
    info['portfolio_valuation']


In [ ]:

plt.plot(eval_profit)
np.mean(eval_profit)

In [ ]:
test_env.max_episode_duration=148

In [ ]:
obs,info=test_env.reset()
action=None
reward=0
terminated = truncated = False

while not terminated and not truncated:

    action = agent.compute_single_action(obs,explore=False)
    # action = train_env.action_space.sample()
    # print(action)
    next_obs, reward, terminated, truncated, info = test_env.step(action)

    obs=next_obs


info['portfolio_valuation']

In [ ]:
test_env.save_for_render(dir = "PPO_test_render_logs")

In [ ]:
renderer = Renderer(render_logs_dir="PPO_test_render_logs")

In [ ]:
# Add Custom Metrics (Annualized metrics)
renderer.add_metric(
    name = "Annual Market Return",
    function = lambda df : f"{ ((df['close'].iloc[-1] / df['close'].iloc[0])**(pd.Timedelta(days=365)/(df.index.values[-1] - df.index.values[0]))-1)*100:0.2f}%"
)
renderer.add_metric(
        name = "Annual Portfolio Return",
        function = lambda df : f"{((df['portfolio_valuation'].iloc[-1] / df['portfolio_valuation'].iloc[0])**(pd.Timedelta(days=365)/(df.index.values[-1] - df.index.values[0]))-1)*100:0.2f}%"
)

renderer.run()